In [1]:
import os
import re
import sys
import ast
import shutil
import librosa
import pandas as pd

from tqdm import tqdm
from collections import Counter
from string import ascii_lowercase

In [2]:
sys.path.append("..")

In [3]:
from utils import transform_audio_file, recursive_search, create_dir, delete_dir

In [4]:
prj_path = os.path.dirname(os.getcwd())
data_path = os.path.join(prj_path, 'data')
voxforge_data_path = os.path.join(data_path, 'voxforge')
librivox_data_path = os.path.join(data_path, 'librivox')

## Prepare data

### Audio data

In [5]:
audio_librivox = recursive_search(os.path.join(librivox_data_path, 'audio'))
audio_librivox['duration'] = audio_librivox['file_path'].map(lambda x: librosa.get_duration(filename=x))
audio_librivox['file_size'] = audio_librivox['file_path'].map(lambda x: os.path.getsize(x))

In [6]:
print(audio_librivox.shape)
print(f"Total duration: {audio_librivox['duration'].sum()/60/60:.2f} hours")
print(f"Total size: {audio_librivox['file_size'].sum()/1024/1024:.2f} mb")
audio_librivox.sample(5)

(34193, 4)
Total duration: 83.23 hours
Total size: 4573.50 mb


,file_name,file_path,duration,file_size
30772,obruchov__dorogoyu_tsinoyu_s000444.wav,/home/dima/Projects/stt_uk/data/librivox/audio...,11.489875,183882
26477,miskun__15YO_Capitan_s003951.wav,/home/dima/Projects/stt_uk/data/librivox/audio...,7.174875,114842
23807,shepel__zvirobij_s006111.wav,/home/dima/Projects/stt_uk/data/librivox/audio...,9.080875,145338
24961,miskun__15YO_Capitan_s004099.wav,/home/dima/Projects/stt_uk/data/librivox/audio...,6.572750,105208
32533,obruchov__voly_29_f000059.wav,/home/dima/Projects/stt_uk/data/librivox/audio...,5.639875,90282


In [7]:
audio_voxforge = recursive_search(os.path.join(voxforge_data_path, 'audio'))
audio_voxforge['duration'] = audio_voxforge['file_path'].map(lambda x: librosa.get_duration(filename=x))
audio_voxforge['file_size'] = audio_voxforge['file_path'].map(lambda x: os.path.getsize(x))

In [8]:
print(audio_voxforge.shape)
print(f"Total duration: {audio_voxforge['duration'].sum()/60/60:.2f} hours")
print(f"Total size: {audio_voxforge['file_size'].sum()/1024/1024:.2f} mb")
audio_voxforge.sample(5)

(390, 4)
Total duration: 1.00 hours
Total size: 55.10 mb


,file_name,file_path,duration,file_size
375,Alias-20160403-lmx__uk_0029.wav,/home/dima/Projects/stt_uk/data/voxforge/audio...,12.625,202044
184,dmitriytim-20160401-nga__uk_0025.wav,/home/dima/Projects/stt_uk/data/voxforge/audio...,6.000,96044
170,anonymous-20150723-mom__uk_0046.wav,/home/dima/Projects/stt_uk/data/voxforge/audio...,11.425,182844
87,anonymous-20131219-pxl__uk_0032.wav,/home/dima/Projects/stt_uk/data/voxforge/audio...,13.375,214044
323,anonymous-20131222-pmk__uk_0011.wav,/home/dima/Projects/stt_uk/data/voxforge/audio...,6.125,98044


### gender

In [9]:
audio_df = pd.concat([audio_librivox, audio_voxforge])
audio_df['speaker_id'] = audio_df['file_name'].map(lambda x: x.split("__", 1)[0])

In [10]:
speaker_gender = {item: 'm' for item in audio_df['speaker_id'].unique()}

for item in ['Anna-20160402-kxf', 'sumska', 'Darrr-20170412-jau']:
    speaker_gender[item] = 'f'

audio_df['gender'] = audio_df['speaker_id'].map(speaker_gender)

In [11]:
print(audio_df.shape)
audio_df.sample(5)

(34583, 6)


,file_name,file_path,duration,file_size,speaker_id,gender
29949,obruchov__voly_16_f000063.wav,/home/dima/Projects/stt_uk/data/librivox/audio...,11.949875,191242,obruchov,m
31923,obruchov__voly_23_f000222.wav,/home/dima/Projects/stt_uk/data/librivox/audio...,5.779875,92522,obruchov,m
30315,miskun__15YO_Capitan_s001947.wav,/home/dima/Projects/stt_uk/data/librivox/audio...,7.689875,123082,miskun,m
26983,shepel__zvirobij_s006968.wav,/home/dima/Projects/stt_uk/data/librivox/audio...,5.699875,91242,shepel,m
9960,miskun__15YO_Capitan_s001689.wav,/home/dima/Projects/stt_uk/data/librivox/audio...,9.964875,159482,miskun,m


### text

In [12]:
with open(os.path.join(voxforge_data_path, 'prompts.txt')) as file_:
    lst = [item.split(" ", 1) for item in file_.readlines()]
text_voxforge = pd.DataFrame(lst)
text_voxforge[2] = 'voxforge'

In [13]:
with open(os.path.join(librivox_data_path, 'prompts.txt')) as file_:
    lst = [item.split(" ", 1) for item in file_.readlines()]
text_librivox = pd.DataFrame(lst)
text_librivox[2] = 'librivox'

In [14]:
text_df = pd.concat([text_librivox, text_voxforge])
text_df.columns = ['file_name', 'transcript', 'source']

In [15]:
print(text_df.shape)
text_df.head()

(34579, 3)


,file_name,transcript,source
0,obruchov__tini_zabutyh_predkiv_s000001.wav,"Я ще заграю до танцю,- бадьорив він чугайстра ...",librivox
1,obruchov__tini_zabutyh_predkiv_s000002.wav,"Вони, здається, гойдалися з нею ще у колисці, ...",librivox
2,obruchov__tini_zabutyh_predkiv_s000003.wav,"І вона співанками косичила їх розлучення, Їй б...",librivox
3,obruchov__tini_zabutyh_predkiv_s000004.wav,"Іду, Марічко! - билась в Іванових грудях одпов...",librivox
4,obruchov__tini_zabutyh_predkiv_s000005.wav,Засідали за мережаний стіл. тяжкі в своїм овеч...,librivox


#### preprocess text

In [16]:
letters_to_replace = {"i": "і", "a": "а", "o": "о", "y": "у", "e": "е", 
                      "p": "р", "n": "п", "c": "с", "x": "х", "r": "г",
                      "m": "м", "h": "н", "b": "в", "t": "т"}

In [17]:
def clean_text(x):
    x = x.replace('laissez donc le domestique ecoute', 'люсі дунк лю домєстік екют')
    for k, v in letters_to_replace.items():
        x = x.replace(k, v)
    return x


def remove_digits(x):
    x = re.sub(r"\d+", " ", x)
    return x


def tokenize(x):
    sentences = nlp_uk(x).sentences
    sents = []
    for sentence in sentences:
        res = [token.text if token.upos not in ('PUNCT', 'NUM') else " " for token in sentence.words ]
        sents.append(res)
    sents = [" ".join(remove_digits(" ".join(x)).lower().split()) for x in sents]
    return sents

In [18]:
try:
    text_df = pd.read_csv(os.path.join(data_path, 'text_df.csv')).fillna('')
    text_df['transcript_clean_lst'] = text_df['transcript_clean_lst'].map(ast.literal_eval)
except FileNotFoundError:
    import stanfordnlp
    import warnings

    warnings.filterwarnings("ignore")
    nlp_uk = stanfordnlp.Pipeline(lang='uk')
    
    text_df['transcript_clean'] = text_df.transcript.map(clean_text)

    lst = []
    for item in tqdm(text_df['transcript_clean'].values):
        lst.append(tokenize(item))
    text_df['transcript_clean_lst'] = lst

    text_df['transcript_clean'] = text_df['transcript_clean_lst'].map(lambda x: " ".join(x))
    text_df.to_csv(os.path.join(data_path, 'text_df.csv'), index=False)

In [19]:
print(text_df.shape)
text_df.head()

(34579, 5)


,file_name,transcript,source,transcript_clean,transcript_clean_lst
0,obruchov__tini_zabutyh_predkiv_s000001.wav,"Я ще заграю до танцю,- бадьорив він чугайстра ...",librivox,я ще заграю до танцю бадьорив він чугайстра й ...,[я ще заграю до танцю бадьорив він чугайстра й...
1,obruchov__tini_zabutyh_predkiv_s000002.wav,"Вони, здається, гойдалися з нею ще у колисці, ...",librivox,вони здається гойдалися з нею ще у колисці хлю...,[вони здається гойдалися з нею ще у колисці хл...
2,obruchov__tini_zabutyh_predkiv_s000003.wav,"І вона співанками косичила їх розлучення, Їй б...",librivox,і вона співанками косичила їх розлучення їй бу...,[і вона співанками косичила їх розлучення їй б...
3,obruchov__tini_zabutyh_predkiv_s000004.wav,"Іду, Марічко! - билась в Іванових грудях одпов...",librivox,іду марічко билась в іванових грудях одповідь ...,[іду марічко билась в іванових грудях одповідь...
4,obruchov__tini_zabutyh_predkiv_s000005.wav,Засідали за мережаний стіл. тяжкі в своїм овеч...,librivox,засідали за мережаний стіл тяжкі в своїм овечі...,[засідали за мережаний стіл тяжкі в своїм овеч...


In [20]:
words = []

for item in tqdm(text_df["transcript_clean"].str.split().values):
    words.extend(item)

100%|██████████| 34579/34579 [00:00<00:00, 2565989.14it/s]


In [21]:
Counter(words).most_common(10)

[('і', 12612),
 ('не', 12347),
 ('на', 11494),
 ('що', 8306),
 ('в', 7737),
 ('з', 7642),
 ('й', 7467),
 ('а', 6416),
 ('до', 5381),
 ('та', 5327)]

In [22]:
unique_words = list(set(words))
print(len(unique_words))

73211


In [23]:
skip = False
bad_word = []

for item in unique_words:
    skip = False
    for letter in ascii_lowercase:
        if skip:
            break
        if letter in item:
            bad_word.append(item)
            skip = True
len(bad_word)

31

In [24]:
bad_word

['ситуацiя',
 'hех',
 'артистiв',
 'xii',
 'тiльки',
 'a',
 'полiтикою',
 'eх',
 'xіба',
 'dопс',
 'iнша',
 'кj',
 'v',
 'xоч',
 'есоuте',
 'i',
 'hепосиді',
 'bін',
 'bсі',
 'xv',
 'vоlу',
 'аvапті',
 'першi',
 'mотрі',
 'mишачий',
 'mемепто',
 'полiтики',
 'lаіssеz',
 'dомеsтіquе',
 'розумiю',
 'bірує']

In [25]:
mask = text_df.transcript_clean.apply(lambda x: any(item for item in bad_word if item in x))
text_df = text_df[~mask]

In [26]:
print(text_df.shape)
text_df.head()

(34512, 5)


,file_name,transcript,source,transcript_clean,transcript_clean_lst
0,obruchov__tini_zabutyh_predkiv_s000001.wav,"Я ще заграю до танцю,- бадьорив він чугайстра ...",librivox,я ще заграю до танцю бадьорив він чугайстра й ...,[я ще заграю до танцю бадьорив він чугайстра й...
1,obruchov__tini_zabutyh_predkiv_s000002.wav,"Вони, здається, гойдалися з нею ще у колисці, ...",librivox,вони здається гойдалися з нею ще у колисці хлю...,[вони здається гойдалися з нею ще у колисці хл...
2,obruchov__tini_zabutyh_predkiv_s000003.wav,"І вона співанками косичила їх розлучення, Їй б...",librivox,і вона співанками косичила їх розлучення їй бу...,[і вона співанками косичила їх розлучення їй б...
3,obruchov__tini_zabutyh_predkiv_s000004.wav,"Іду, Марічко! - билась в Іванових грудях одпов...",librivox,іду марічко билась в іванових грудях одповідь ...,[іду марічко билась в іванових грудях одповідь...
4,obruchov__tini_zabutyh_predkiv_s000005.wav,Засідали за мережаний стіл. тяжкі в своїм овеч...,librivox,засідали за мережаний стіл тяжкі в своїм овечі...,[засідали за мережаний стіл тяжкі в своїм овеч...


### Final df

In [27]:
df = audio_df.merge(text_df, on='file_name')

In [28]:
print(df.shape)
df.head()

(34512, 10)


,file_name,file_path,duration,file_size,speaker_id,gender,transcript,source,transcript_clean,transcript_clean_lst
0,miskun__15YO_Capitan_s003791.wav,/home/dima/Projects/stt_uk/data/librivox/audio...,7.879875,126122,miskun,m,"Вітер чимдалі дужчав, однак не змінював свого ...",librivox,вітер чимдалі дужчав однак не змінював свого н...,[вітер чимдалі дужчав однак не змінював свого ...
1,loboda__chorna_rada_s002224.wav,/home/dima/Projects/stt_uk/data/librivox/audio...,3.417875,54730,loboda,m,"Ну, прощайте ж, братці, навіки!\n",librivox,ну прощайте ж братці навіки,[ну прощайте ж братці навіки]
2,loboda__zahar_berkut_s000726.wav,/home/dima/Projects/stt_uk/data/librivox/audio...,4.769875,76362,loboda,m,котра ген-ген сходилася з долиною Стрия.\n,librivox,котра ген- ген сходилася з долиною стрия,[котра ген- ген сходилася з долиною стрия]
3,obruchov__dorogoyu_tsinoyu_s000477.wav,/home/dima/Projects/stt_uk/data/librivox/audio...,15.389875,246282,obruchov,m,"Напоєний незабаром зіллям, із перев'язаною ран...",librivox,напоєний незабаром зіллям із перев'язаною рано...,[напоєний незабаром зіллям із перев'язаною ран...
4,shepel__zvirobij_s003180.wav,/home/dima/Projects/stt_uk/data/librivox/audio...,8.661875,138634,shepel,m,"Ніхто не знав, за яких обставин її влучено: ма...",librivox,ніхто не знав за яких обставин її влучено мабу...,[ніхто не знав за яких обставин її влучено маб...


## Kaldi project requirements

In [29]:
proj_name = 'stt_uk'

kaldi_path = f'/home/{os.environ.get("USER")}/kaldi'
# set here correct path when necessary
kaldi_proj_path = os.path.join(kaldi_path, 'egs', proj_name, 's5')

In [30]:
kaldi_proj_path

'/home/dima/kaldi/egs/stt_uk/s5'

In [31]:
if not os.path.exists(kaldi_path):
    print("KALDI IS ABSENT!!!!!")

In [32]:
DELETE_ON_CREATING = False

folders = ["audio", "audio/train", "audio/test", 
           "data", "data/train", "data/test", "data/local", "data/local/dict", 
           "conf"]

if DELETE_ON_CREATING:
    delete_dir(kaldi_proj_path)

for folder in folders:
    create_dir(os.path.join(kaldi_proj_path, folder))

#### audio data

In [33]:
# train, test = df.loc[df.source != 'voxforge'], df.loc[df.source == 'voxforge']
train, test = df, df.loc[df.source == 'voxforge']

print(train.shape[0], test.shape[0])

34512 375


In [34]:
# test = test[test.speaker_id == 'Anna-20160402-kxf']
# train = test

In [35]:
# TODO: fix test on train

In [36]:
for df, destination in [(train, 'train'), (test, 'test')]:
    for spkr in tqdm(df.speaker_id.unique()):
        create_dir(os.path.join(kaldi_proj_path, 'audio', destination, spkr))
        
        for row in df.loc[df.speaker_id == spkr].iterrows():
            shutil.copy2(row[1].file_path, 
                         os.path.join(kaldi_proj_path, 'audio', destination, spkr, row[1].file_name))

100%|██████████| 39/39 [00:00<00:00, 122.77it/s]


#### spk2gender

In [37]:
train[["speaker_id", "gender"]].drop_duplicates().to_csv(
    os.path.join(kaldi_proj_path, 'data', 'train', 'spk2gender'), 
    sep=" ", index=False, header=None)
test[["speaker_id", "gender"]].drop_duplicates().to_csv(
    os.path.join(kaldi_proj_path, 'data', 'test', 'spk2gender'), 
    sep=" ", index=False, header=None)

#### wav.scp

In [38]:
train["path"] = train.apply(lambda x: f"{kaldi_proj_path}/audio/train/{x.speaker_id}/{x.file_name}", 1)
test["path"] = test.apply(lambda x: f"{kaldi_proj_path}/audio/test/{x.speaker_id}/{x.file_name}", 1)

/home/dima/anaconda3/envs/p36/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [39]:
train[["file_name", "path"]].to_csv(os.path.join(kaldi_proj_path, 'data', 'train', 'wav.scp'), 
                                    sep=" ", index=False, header=None)
test[["file_name", "path"]].to_csv(os.path.join(kaldi_proj_path, 'data', 'test', 'wav.scp'), 
                                   sep=" ", index=False, header=None)

#### text

In [40]:
with open(os.path.join(kaldi_proj_path, 'data', 'train', 'text'), "w") as file_:
    for line in (train["file_name"] + " " + train["transcript_clean"]).values:
        file_.write(line.replace('-', ' ').strip() + "\n")
        
with open(os.path.join(kaldi_proj_path, 'data', 'test', 'text'), "w") as file_:
    for line in (test["file_name"] + " " + test["transcript_clean"]).values:
        file_.write(line.replace('-', ' ').strip() + "\n")

#### utt2spk

In [41]:
train[["file_name", "speaker_id"]].to_csv(os.path.join(kaldi_proj_path, 'data', 'train', 'utt2spk'), 
                                          sep=" ", index=False, header=None)
test[["file_name", "speaker_id"]].to_csv(os.path.join(kaldi_proj_path, 'data', 'test', 'utt2spk'), sep=" ", index=False, header=None)

#### corpus.txt

In [47]:
sentences = []

for item in df.transcript_clean_lst.values:
    for i in item:
        sentences.append(i)

In [43]:
try:
    with open(os.path.join(data_path, 'ner-uk-corpus-new.txt')) as file_:
        texts = file_.readlines()
except FileNotFoundError:
    import stanfordnlp
    import warnings

    warnings.filterwarnings("ignore")
    nlp_uk = stanfordnlp.Pipeline(lang='uk')
    
    with open(os.path.join(data_path, 'ner-uk-corpus.txt')) as file_:
        texts = file_.readlines()

    lst = []
    for item in tqdm(texts):
        lst.append(tokenize(item))
    
    with open(os.path.join(data_path, 'ner-uk-corpus-new.txt'), 'w') as file_:
        for item in lst:
            for it in item:
                file_.write(it.strip() + "\n")
    texts = lst

In [44]:
with open(os.path.join(kaldi_proj_path, 'data', 'local', 'corpus.txt'), "w") as file_:
    for line in sentences + texts:
        file_.write(line.replace('-', ' ').strip() + "\n")

In [52]:
df.shape

(375, 11)

#### lexicon.txt

In [97]:
sil_phones = [("!SIL", "sil"), ("<UNK>", "spn")]

phoneme preparation mechanism may be different

In [99]:
len(set(" ".join(texts).split()))

49945

In [101]:
# words = []

# for item in tqdm(text_df["transcript_clean"].str.split().values):
#     words.extend(item)
# unique_words = list(set(words))

# with open('lexicon_prep1.txt', "w") as file_:
#     for line in set(" ".join(texts).split()):
#         file_.write(line.strip() + "\n")

In [134]:
with open(os.path.join(data_path, 'lexicon_prep_out.txt')) as file_:
    lexicon1 = file_.readlines()
    
with open(os.path.join(data_path, 'lexicon_prep_out1.txt')) as file_:
    lexicon2 = file_.readlines()
    
lexicon_phones = [tuple(item.strip().split(" ", 1)) for item in lexicon1 + lexicon2]

In [135]:
lexicon_phones = [item for item in lexicon_phones if len(item) == 2]

In [136]:
with open(os.path.join(kaldi_proj_path, 'data', 'local', "dict", 'lexicon.txt'), "w") as file_:
    for word, phones in sil_phones + lexicon_phones:
        file_.write(word + " " + " ".join(phones.split(" ")) + "\n")

#### nonsilence_phones.txt

In [137]:
unique_phones = ["а"]

for k, v in lexicon_phones:
    unique_phones.extend(v.split(" "))

unique_phones = list(sorted(set(unique_phones)))
len(unique_phones)

82

In [138]:
with open(os.path.join(kaldi_proj_path, 'data', 'local', "dict", 'nonsilence_phones.txt'), "w") as file_:
    for phone in sorted(unique_phones):
        file_.write(phone + "\n")

#### silence_phones.txt

In [139]:
with open(os.path.join(kaldi_proj_path, 'data', 'local', "dict", 'silence_phones.txt'), "w") as file_:
    for phone in sorted(list(set(" ".join([item[1] for item in sil_phones]).split()))):
        file_.write(phone + "\n")

#### optional_silence.txt

In [140]:
with open(os.path.join(kaldi_proj_path, 'data', 'local', "dict", 'optional_silence.txt'), "w") as file_:
    for phone in ["sil"]:
        file_.write(phone + "\n")

### copy all required files to project dir

In [141]:
misc_path = os.path.join(prj_path, 'kaldi', 'misc')
conf_path = os.path.join(misc_path, 'conf')

In [142]:
for item in os.listdir(conf_path):
    shutil.copy2(os.path.join(conf_path, item), 
                 os.path.join(kaldi_proj_path, 'conf', item))

In [143]:
for item in ['cmd.sh', 'run.sh', 'path.sh']:
    shutil.copy2(os.path.join(misc_path, item), 
                 os.path.join(kaldi_proj_path, item))

In [144]:
! cp -R /home/$USER/kaldi/egs/stt_uk/s5/data/local/dict/* /home/$USER/kaldi/egs/stt_uk/s5/data/local/

make sure that here is a correct project path

In [145]:
# ! rm -rf /home/$USER/kaldi/egs/stt_uk/s5/steps /home/$USER/kaldi/egs/stt_uk/s5/utils /home/$USER/kaldi/egs/stt_uk/s5/local/

! cp -R /home/$USER/kaldi/egs/babel/s5d/steps /home/$USER/kaldi/egs/stt_uk/s5/steps
! cp -R /home/$USER/kaldi/egs/babel/s5d/utils /home/$USER/kaldi/egs/stt_uk/s5/utils
! cp -R /home/$USER/kaldi/egs/babel/s5d/local/ /home/$USER/kaldi/egs/stt_uk/s5/local/

In [146]:
! cp -R /home/$USER/kaldi/egs/wsj/s5/local/score.sh /home/$USER/kaldi/egs/stt_uk/s5/local/score.sh